In [1]:
## Preloads
using Statistics
using FFTW
using Plots
using BenchmarkTools
using Profile
using LinearAlgebra
using Measures
using HDF5
push!(LOAD_PATH, pwd())
using DHC_2DUtils
using MLDatasets
using Images
theme(:juno)
using DSP
using Interpolations
using Distributed
using ProgressMeter

  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:176
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:179
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:181
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:184
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:186
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:189
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:203
  likely near /n/home12/saydjari/.julia/packages/Plots

In [2]:
train_angles =range(π/24,stop=π,length=24)

0.1308996938995747:0.1308996938995747:3.141592653589793

In [3]:
test_angles = range(π/48,stop=π,length=48)

0.06544984694978735:0.06544984694978735:3.141592653589793

In [4]:
workers()

1-element Array{Int64,1}:
 1

In [5]:
addprocs(30)

30-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31

In [6]:
train_x, train_y = MNIST.traindata()
test_x, test_y   = MNIST.testdata()

lst_train = Array{Any}(undef, 0)
for i = 1:60000
    push!(lst_train,train_x[:,:,i])
end


lst_test = Array{Any}(undef, 0)
for i = 1:10000
    push!(lst_test,test_x[:,:,i])
end

Ok, so the new flow is to call a wrapper that does the above

In [7]:
function paraWrapper(filename,dataname,coeffdim,iter_prod,batch_size,n_batch)
    fid = h5open(filename,"w")
    dset = create_dataset(fid, dataname, datatype(Float64), dataspace(coeffdim,length(iter_prod)), chunk=(coeffdim,batch_size),deflate=3)
    close(fid)
    @showprogress for i=1:n_batch
        subset = Iterators.take(iter_prod,batch_size)
        DHC_out = pmap(mnist_DHC,subset)
        DHC_out = hcat(DHC_out...)

        fid = h5open(filename,"r+")
        data = open_dataset(fid, dataname)
        data[:,(1:batch_size).+(i-1)*batch_size] = DHC_out
        close(fid)
        iter_prod = Iterators.drop(iter_prod,batch_size)
    end
end

paraWrapper (generic function with 1 method)

In [21]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 200
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,4,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

7200

In [10]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,4,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

9600

In [11]:
paraWrapper("mnist_DHC_test_eq_L4_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L4_s.h5", "main/angles", collect(test_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:11:39


In [12]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,4,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

28800

In [13]:
paraWrapper("mnist_DHC_train_eq_L4_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L4_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:34:15


In [14]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

9600

In [15]:
paraWrapper("mnist_DHC_test_eq_L8_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L8_s.h5", "main/angles", collect(test_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:21:26


In [16]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

14400

In [17]:
paraWrapper("mnist_DHC_train_eq_L8_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L8_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 1:04:13


In [18]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,12,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

4800

In [19]:
paraWrapper("mnist_DHC_test_eq_L12_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L12_s.h5", "main/angles", collect(test_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:33:57


In [20]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,12,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

14400

In [21]:
paraWrapper("mnist_DHC_train_eq_L12_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L12_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 1:44:40


In [22]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,16,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L16_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

Progress: 100%|█████████████████████████████████████████| Time: 0:51:05


LoadError: syntax: missing comma or ) in argument list

In [8]:
h5write("mnist_DHC_test_eq_L16_s.h5", "main/angles", collect(test_angles))

In [8]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 200
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,16,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L16_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L16_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 2:37:35


In [9]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 200
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,32,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L32_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L32_s.h5", "main/angles", collect(test_angles))

Progress: 100%|█████████████████████████████████████████| Time: 2:23:20
Progress:   1%|▍                                        |  ETA: 7:24:26

InterruptException: InterruptException:

In [8]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 200
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,32,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L32_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L32_s.h5", "main/angles", collect(train_angles))

RemoteException: InterruptException:
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:851
deserialize_fillarray!(::Array{Any,1}, ::Distributed.ClusterSerializer{Sockets.TCPSocket}) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:1094
deserialize_array(::Distributed.ClusterSerializer{Sockets.TCPSocket}) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:1086
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:773
deserialize at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:722 [inlined]
deserialize_msg(::Distributed.ClusterSerializer{Sockets.TCPSock

In [10]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,6,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L6_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L6_s.h5", "main/angles", collect(test_angles))
        
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,6,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L6_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L6_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:17:24
Progress: 100%|█████████████████████████████████████████| Time: 0:54:09


In [11]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,3,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L3_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L3_s.h5", "main/angles", collect(test_angles))
        
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,3,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L3_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L3_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:10:56
Progress: 100%|█████████████████████████████████████████| Time: 0:33:12


In [ ]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,5,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L5_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L5_s.h5", "main/angles", collect(test_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:15:55
Progress:  74%|██████████████████████████████▍          |  ETA: 0:12:32

In [9]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,5,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L5_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L5_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:44:37


In [10]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,7,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L7_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L7_s.h5", "main/angles", collect(test_angles))
        
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 200
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,7,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L7_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L7_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:20:20
Progress: 100%|█████████████████████████████████████████| Time: 0:56:38


In [11]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,9,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L9_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L9_s.h5", "main/angles", collect(test_angles))
        
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 200
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,9,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L9_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L9_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:26:17
Progress: 100%|█████████████████████████████████████████| Time: 1:21:30


In [12]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,10,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

paraWrapper("mnist_DHC_test_eq_L10_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L10_s.h5", "main/angles", collect(test_angles))
        
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 200
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,10,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size
        
paraWrapper("mnist_DHC_train_eq_L10_s.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L10_s.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 0:29:26
Progress: 100%|█████████████████████████████████████████| Time: 1:33:14
